In [1]:
import oracledb
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:


oracledb.init_oracle_client(lib_dir="C:/Users/3calss_15/Downloads/instantclient_11_2")  # Instant Client의 경로로 바꿔주세요
# 오라클 DB 연결 설정
connection = oracledb.connect(user='mainproject', password='mainproject', dsn='localhost:1521/xe')

# SQL 쿼리로 데이터베이스에서 필요한 데이터 불러오기
def fetch_data():
    query = """
    SELECT p.product_id, p.product_name, p.category_id, p.brand, p.price, p.stock_quantity
    FROM products p
    """
    products_df = pd.read_sql(query, con=connection)

    query_user_views = """
    SELECT uv.user_id, uv.product_id 
    FROM user_views uv
    """
    user_views_df = pd.read_sql(query_user_views, con=connection)

    query_cart_history = """
    SELECT ch.user_id, ch.product_id
    FROM cart_history ch
    """
    cart_history_df = pd.read_sql(query_cart_history, con=connection)
      
    return products_df, user_views_df, cart_history_df


In [23]:
# 데이터 전처리 및 모델 생성
def preprocess_and_recommend(user_id, products_df, user_views_df, cart_history_df):
    # 제품 속성 결합 (카테고리, 브랜드, 설명 등)
    products_df['combined_features'] = products_df['category_id'].astype(str) + ' ' + products_df['brand']
    # 'category_id'와 'brand' 컬럼의 값 출력
    print(products_df[['category_id', 'brand']].head())  # 상위 5개의 데이터를 출력

    # TF-IDF 기반으로 각 제품의 특징 벡터화
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(products_df['combined_features'])

    # 코사인 유사도를 사용하여 제품 간 유사도 계산
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # 사용자가 조회한 제품 및 장바구니 내역을 합치기
    user_interactions = pd.concat([
        user_views_df[user_views_df['user_id'] == user_id],
        cart_history_df[cart_history_df['user_id'] == user_id]
    ])

    # 사용자가 상호작용한 제품의 ID 리스트
    user_products = user_interactions['product_id'].unique()

    # 사용자가 상호작용한 제품과 유사한 제품 추천
    product_indices = products_df[products_df['product_id'].isin(user_products)].index
    sim_scores = list(enumerate(cosine_sim[product_indices].mean(axis=0)))

    # 유사도에 따라 제품 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 10개의 추천 제품 ID 추출
    top_product_indices = [i[0] for i in sim_scores[:10]]
    recommended_products = products_df.iloc[top_product_indices]
    
    return recommended_products


In [24]:
# 메인 함수
def main(user_id):
    products_df, user_views_df, cart_history_df = fetch_data()
    #print(products_df)
    #print(user_views_df)
    #print(cart_history_df)  
    recommendations = preprocess_and_recommend(user_id, products_df, user_views_df, cart_history_df)

    print("추천 상품 목록:")
    print(recommendations[['product_id', 'product_name', 'brand', 'price']])

# 예시 실행 (user_id가 1인 사용자에게 추천)
main(7)


     PRODUCT_ID                        PRODUCT_NAME  CATEGORY_ID  BRAND  \
0             1                             로지텍G102            1    로지텍   
1             2                        AULA F99 PRO            2   AULA   
2             3             DELL UltraSharp U2723QE            3   DELL   
3             4             Razer DeathAdder V2 Pro            1    레이저   
4             5            MAXTILL MAXTILL TRON G10            1    맥스틸   
..          ...                                 ...          ...    ...   
130         131  에이투 플로우 RGB 레인보우 블루투스 5.3 게이밍 사운드바            5    에이투   
131         132            앱코 저소음 텐키리스 멤브레인 87키 키보드            2     앱코   
132         133     홈플래닛 방송용/수업용 HD웹캠 화상카메라 (마이크내장)            4   홈플래닛   
133         134            아이플렉스 토끼 무선 마우스 WM-RABIT            1  아이플렉스   
134         135          앱코 가상 7.1 RGB 게이밍 헤드셋 N460            6     앱코   

      PRICE  STOCK_QUANTITY  
0     19900               7  
1     65900              98  
2    6890

C:\Users\3calss_15\AppData\Local\Temp\ipykernel_3420\3551240973.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products_df = pd.read_sql(query, con=connection)
C:\Users\3calss_15\AppData\Local\Temp\ipykernel_3420\3551240973.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_views_df = pd.read_sql(query_user_views, con=connection)
C:\Users\3calss_15\AppData\Local\Temp\ipykernel_3420\3551240973.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cart_history_df = pd.read_sql(query_cart_history, con=connection)


KeyError: 'category_id'